[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=08%20graphs%20-%2002%20computation.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Graphs in Julia with Graphs.jl

a 101 introduction session

In [ ]:
using Random; Random.seed!(2022);  # make sure this tutorial is reproducible

# Graphs.jl - Computations

"The project goal is to mirror the functionality of robust network and graph analysis libraries such as NetworkX." (https://github.com/JuliaGraphs/Graphs.jl)

With `Graph` you can do many advanced computations which we are going to look into now 

In [ ]:
using Graphs, GraphPlot, Plots

# Full example - global cascades on random networks

Simple simulation how easy it for a post to go viral in a network. Known as Watts-model.
Adapted from here https://nbviewer.org/github/JuliaGraphs/JuliaGraphsTutorials/blob/master/Watts-Model.ipynb

In [ ]:
"""
Computes the fraction of neighbors engaged within the neighborhood
of a given node.
"""
function fraction_engaged(node::Int,
                          G::Graph,
                          engagement::BitVector)
    num_engaged_neighbors = 0
    for nbr in neighbors(G, node)
        if engagement[nbr] == true
            num_engaged_neighbors += 1
        end
    end
    return num_engaged_neighbors / length(neighbors(G, node))
end

In [ ]:
"""
Updates the engagement of all vertices, one iteration only.
"""
function update_engagement!(G::Graph,
                            engagement::BitVector,
                            threshold::Float64)
    for node in Random.shuffle(vertices(G))
        if engagement[node] == false
            if fraction_engaged(node, G, engagement) > threshold
                engagement[node] = true
            end
        end
    end

    return nothing
end

In [ ]:
import StatsBase
"""
Executes the simulation

Output
-----------
A vector of number of engaged nodes at the end of each realization
of the simulation

Hyper Parameters of the model
----------
1. Number of nodes in the Watts-Strogatz graph (n)
2. Average degree (z)
3. Threshold (a specific value)
4. Time steps for simulation to be run (T)
5. Number of realizations
"""
function simulation(; n::Int, z::Int, threshold::Float64, T::Int, n_realizations::Int)
    output = Vector{Int}(undef, n_realizations)
    beta = z/n

    for r in 1:n_realizations
        G = watts_strogatz(n, z, beta)
        # Select a single random node from the network and seed it
        engagement = falses(nv(G))
        engagement[StatsBase.sample(vertices(G))] = true

        # Update the network for predefined number of time steps
        for _ in 1:T
            update_engagement!(G, engagement, threshold)
        end
        output[r] = sum(engagement)
    end

    return output
end

In [ ]:
n = 10^4
z = 5
threshold = 0.18
T = 50
n_realizations = 100

In [ ]:
# !!CAUTION!!, this is a big plot which takes time to compute
# and even after slows down your webpage significantly
gplot(watts_strogatz(n, z, z/n))

In [ ]:
data1 = simulation(; n, z, threshold, T, n_realizations)
histogram(data1, xlab="Number of engaged nodes", ylab="Frequency", legend=false)

In [ ]:
z2 = 6;  # +1

In [ ]:
data2 = simulation(; n, z=z2, threshold, T, n_realizations);
histogram(data2, xlab="Number of engaged nodes", ylab="Frequency", legend=false)

As can be seen from these two graphs, no global cascade occurs in the second case, while there are a few in the first! It is remarkable that just increasing the average degree of the network by 1 changes the entire outcome of the diffusion process.

The code presented here can be used to reproduce all the results discussed in Watts (2002).

## 💪 it is your turn: Play with the above hyperparameters

and see how the results change

# Predefined algorithms 

Provided by Graphs.jl:
- [Path traversal](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/pathing/)
- [Coloring](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/coloring/)
- [Distance](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/distance/)
- [Centrality measures](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/centrality/)
- [Community structures](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/community/)
- [Graph decomposition](https://docs.juliahub.com/Graphs/VJ6vx/1.4.1/degeneracy/)

Provided by other packages:
- [Matching](https://github.com/JuliaGraphs/GraphsMatching.jl)
- [Network Interdiction](https://github.com/JuliaGraphs/LightGraphsExtras.jl)
- [Partitioning](https://github.com/JuliaSparse/Metis.jl)

# Parallel algorithms

The following is a current list of parallel algorithms.
```
using Graphs
import Graphs.Parallel
```

Centrality measures:
- `Parallel.betweenness_centrality`
- `Parallel.closeness_centrality`
- `Parallel.pagerank`
- `Parallel.radiality_centrality`
- `Parallel.stress_centrality`

Distance measures:
- `Parallel.center`
- `Parallel.diameter`
- `Parallel.eccentricity`
- `Parallel.radius`

Shortest paths algorithms:
- `Parallel.bellman_ford_shortest_paths`
- `Parallel.dijkstra_shortest_paths`
- `Parallel.floyd_warshall_shortest_paths`
- `Paralell.johnson_shortest_paths`

Traversal algorithms:
- `Parallel.bfs`
- `Parallel.greedy_color`


# Further example notebooks

jupyter notebooks with focus on applying Graphs.jl (still referred to as LightGraphs.jl)
- [DAG julia pkgs](https://nbviewer.org/github/JuliaGraphs/JuliaGraphsTutorials/blob/master/DAG-Julia-Pkgs.ipynb)
- [The tipping point](https://nbviewer.org/github/JuliaGraphs/JuliaGraphsTutorials/blob/master/TheTippingPoint.ipynb)
- [Talk of the network](https://nbviewer.org/github/JuliaGraphs/JuliaGraphsTutorials/blob/master/Talk-of-the-network.ipynb)

# Thank you for joining

for questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>